In [0]:
# ♥ Evidence it's your account (screenshot this cell output)
from pyspark.sql import functions as F
print("User (Databricks):", spark.sql("SELECT current_user() AS u").collect()[0].u)
print("Workspace URL:", spark.conf.get("spark.databricks.workspaceUrl", "CE"))


User (Databricks): kalvakuntla.b@northeastern.edu
Workspace URL: dbc-509b9ea5-ed04.cloud.databricks.com


In [0]:
# Create (or use) a working database
spark.sql("CREATE DATABASE IF NOT EXISTS mm_demo_db")
spark.sql("USE mm_demo_db")


DataFrame[]

In [0]:
%sql
SELECT COUNT(*) 
FROM healthcare.default.material_master_1_k;


COUNT(*)
1000


In [0]:
%sql
SELECT current_catalog(), current_schema();

current_catalog(),current_schema()
workspace,mm_demo_db


In [0]:
%sql
USE CATALOG healthcare;
USE SCHEMA default;

In [0]:
%sql
SELECT COUNT(*) 
FROM material_master_1_k;


COUNT(*)
1000


In [0]:
bronze_table = "healthcare.default.material_master_1_k"
df = spark.table(bronze_table)
df.display()


material_id|material_name|category|sub_category|uom|unit_cost|supplier_name|country|plant|status|last_updated|lead_time_days|safety_stock|reorder_level|remarks
M00001|Metal Chair|Raw Material|Metal|L|70.63|Rodriguez
M00002|Plastic All|Raw Material|Plastic|NOS|14.24|Garza Inc|Germany|PlantD|Active|2025-08-21|16|653|167|Discussion own night.
M00003|Plastic Perform|Raw Material|Plastic|MTR|78.58|Santos
M00004|Paint Source|Consumable|Paint|KG|268.58|Peterson-Moore|Japan|PlantA|Obsolete|2025-09-13|11|899|346|Behavior benefit.
M00005|Coolant Show|Consumable|Coolant|KG|114.72|Munoz-Roman|India|PlantA|Active|2025-09-25|5|439|167|By two bad fall pick.
M00006|Tape Answer|Packaging|Tape|NOS|105.54|Ray-Bush|India|PlantA|Obsolete|2025-10-15|7|596|398|Significant world talk.
M00007|Brake Prove|Component|Brake|MTR|494.77|James Group|Brazil|PlantB|Obsolete|2025-10-31|12|913|418|Class great prove reduce.
M00008|Plastic Shoulder|Raw Material|Plastic|KG|201.18|Wong
M00009|Wrap Stand|Packaging|Wrap|MTR|70.68|Hawkins
M00010|Tape Under|Packaging|Tape|NOS|70.03|Clark PLC|Japan|PlantA|Active|2025-10-20|5|206|346|Defense material those poor.


In [0]:
from pyspark.sql import functions as F

bronze_table = "healthcare.default.material_master_1_k"
df = spark.table(bronze_table)

# --- Trim strings & normalize blanks to NULL ---
for c, t in df.dtypes:
    if t == "string":
        df = df.withColumn(c, F.trim(F.col(c)))
        df = df.withColumn(
            c,
            F.when(F.col(c).isin("", "null", "NA", "N/A"), None).otherwise(F.col(c))
        )

# --- Cast obvious numeric columns ---
num_cols = [c for c in df.columns if c.lower() in ("unit_cost",)]
for c in num_cols:
    df = df.withColumn(c, F.regexp_replace(F.col(c), r"[^0-9\.\-]", "").cast("double"))

# --- Parse date columns (if present) ---
if "last_updated" in df.columns:
    df = df.withColumn(
        "last_updated",
        F.coalesce(
            F.to_date("last_updated","yyyy-MM-dd"),
            F.to_date("last_updated","MM/dd/yyyy"),
            F.to_date("last_updated","yyyyMMdd")
        )
    )

# --- Expectation 1: material_id must be present ---
if "material_id" in df.columns:
    df = df.filter(F.col("material_id").isNotNull())

# --- Expectation 2: unit_cost >= 0 (keep NULLs) ---
if "unit_cost" in df.columns:
    df = df.filter(F.col("unit_cost").isNull() | (F.col("unit_cost") >= 0))

# --- Expectation 3 (optional but nice): status in set ---
if "status" in df.columns:
    df = df.filter(F.col("status").isin("Active","Obsolete") | F.col("status").isNull())

# --- Deduplicate full rows ---
df = df.dropDuplicates()

print("Silver candidate rows:", df.count())
df.display()


Silver candidate rows: 1000


material_id|material_name|category|sub_category|uom|unit_cost|supplier_name|country|plant|status|last_updated|lead_time_days|safety_stock|reorder_level|remarks
M00001|Metal Chair|Raw Material|Metal|L|70.63|Rodriguez
M00002|Plastic All|Raw Material|Plastic|NOS|14.24|Garza Inc|Germany|PlantD|Active|2025-08-21|16|653|167|Discussion own night.
M00003|Plastic Perform|Raw Material|Plastic|MTR|78.58|Santos
M00004|Paint Source|Consumable|Paint|KG|268.58|Peterson-Moore|Japan|PlantA|Obsolete|2025-09-13|11|899|346|Behavior benefit.
M00005|Coolant Show|Consumable|Coolant|KG|114.72|Munoz-Roman|India|PlantA|Active|2025-09-25|5|439|167|By two bad fall pick.
M00006|Tape Answer|Packaging|Tape|NOS|105.54|Ray-Bush|India|PlantA|Obsolete|2025-10-15|7|596|398|Significant world talk.
M00007|Brake Prove|Component|Brake|MTR|494.77|James Group|Brazil|PlantB|Obsolete|2025-10-31|12|913|418|Class great prove reduce.
M00008|Plastic Shoulder|Raw Material|Plastic|KG|201.18|Wong
M00009|Wrap Stand|Packaging|Wrap|MTR|70.68|Hawkins
M00010|Tape Under|Packaging|Tape|NOS|70.03|Clark PLC|Japan|PlantA|Active|2025-10-20|5|206|346|Defense material those poor.


In [0]:
%sql
USE CATALOG healthcare;
USE SCHEMA default;


In [0]:
from pyspark.sql import functions as F

bronze_table = "healthcare.default.material_master_1_k"
df = spark.table(bronze_table)

# Trim + normalize blanks
for c, t in df.dtypes:
    if t == "string":
        df = df.withColumn(c, F.trim(F.col(c)))
        df = df.withColumn(c, F.when(F.col(c).isin("", "null", "NA", "N/A"), None).otherwise(F.col(c)))

# Numeric cast
if "unit_cost" in df.columns:
    df = df.withColumn("unit_cost", F.regexp_replace("unit_cost", r"[^0-9\.\-]", "").cast("double"))

# Date parse
if "last_updated" in df.columns:
    df = df.withColumn(
        "last_updated",
        F.coalesce(
            F.to_date("last_updated","yyyy-MM-dd"),
            F.to_date("last_updated","MM/dd/yyyy"),
            F.to_date("last_updated","yyyyMMdd")
        )
    )

# Expectations
if "material_id" in df.columns:
    df = df.filter(F.col("material_id").isNotNull())
if "unit_cost" in df.columns:
    df = df.filter(F.col("unit_cost").isNull() | (F.col("unit_cost") >= 0))
if "status" in df.columns:
    df = df.filter(F.col("status").isin("Active","Obsolete") | F.col("status").isNull())

df = df.dropDuplicates()

print("Silver candidate rows:", df.count())
df.display()


Silver candidate rows: 1000


material_id|material_name|category|sub_category|uom|unit_cost|supplier_name|country|plant|status|last_updated|lead_time_days|safety_stock|reorder_level|remarks
M00001|Metal Chair|Raw Material|Metal|L|70.63|Rodriguez
M00002|Plastic All|Raw Material|Plastic|NOS|14.24|Garza Inc|Germany|PlantD|Active|2025-08-21|16|653|167|Discussion own night.
M00003|Plastic Perform|Raw Material|Plastic|MTR|78.58|Santos
M00004|Paint Source|Consumable|Paint|KG|268.58|Peterson-Moore|Japan|PlantA|Obsolete|2025-09-13|11|899|346|Behavior benefit.
M00005|Coolant Show|Consumable|Coolant|KG|114.72|Munoz-Roman|India|PlantA|Active|2025-09-25|5|439|167|By two bad fall pick.
M00006|Tape Answer|Packaging|Tape|NOS|105.54|Ray-Bush|India|PlantA|Obsolete|2025-10-15|7|596|398|Significant world talk.
M00007|Brake Prove|Component|Brake|MTR|494.77|James Group|Brazil|PlantB|Obsolete|2025-10-31|12|913|418|Class great prove reduce.
M00008|Plastic Shoulder|Raw Material|Plastic|KG|201.18|Wong
M00009|Wrap Stand|Packaging|Wrap|MTR|70.68|Hawkins
M00010|Tape Under|Packaging|Tape|NOS|70.03|Clark PLC|Japan|PlantA|Active|2025-10-20|5|206|346|Defense material those poor.


In [0]:
# This avoids /FileStore entirely
df.write.format("delta").mode("overwrite").saveAsTable("healthcare.default.silver_material_master")


In [0]:
%sql
USE CATALOG healthcare;
USE SCHEMA default;

SELECT COUNT(*) AS silver_count FROM silver_material_master;
SELECT * FROM silver_material_master LIMIT 20;

DESCRIBE DETAIL silver_material_master;  -- shows it’s a managed Delta table (no /FileStore path)


format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,edaa1096-91c7-4c50-b46b-a9ec0de51a4f,healthcare.default.silver_material_master,null,,2025-11-10T23:48:18.966Z,2025-11-10T23:48:25.000Z,List(),List(),1,39970,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
%sql
SELECT * FROM silver_material_master LIMIT 20;
SELECT COUNT(*) FROM silver_material_master;
DESCRIBE DETAIL silver_material_master;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,edaa1096-91c7-4c50-b46b-a9ec0de51a4f,healthcare.default.silver_material_master,null,,2025-11-10T23:48:18.966Z,2025-11-10T23:48:25.000Z,List(),List(),1,39970,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
df = df.filter(F.col("material_id").isNotNull())
df = df.filter(F.col("unit_cost").isNull() | (F.col("unit_cost") >= 0))
df = df.filter(F.col("status").isin("Active","Obsolete") | F.col("status").isNull())


In [0]:
from pyspark.sql import functions as F
print("Databricks user:", spark.sql("select current_user() u").first().u)
print("Workspace URL:", spark.conf.get("spark.databricks.workspaceUrl", "CE"))


Databricks user: kalvakuntla.b@northeastern.edu
Workspace URL: dbc-509b9ea5-ed04.cloud.databricks.com
